In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import *
from tensorflow.keras import Model

In [2]:
train = os.path.join('../input/intel-image-classification/seg_train/seg_train')
test = os.path.join('../input/intel-image-classification/seg_test/seg_test')
pred = os.path.join('../input/intel-image-classification/seg_pred/seg_pred')

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)


train_generator = train_datagen.flow_from_directory(train,  
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')


validation_generator = validation_datagen.flow_from_directory(test,  
                                                              target_size=(150, 150), 
                                                              batch_size=1,  
                                                              class_mode='categorical')

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [4]:
pre_trained_model = tf.keras.applications.InceptionResNetV2(include_top=False,
                                                            weights='../input/inceptionv4-1/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                                            input_shape=(150, 150, 3))

#pre_trained_model.summary()

In [5]:
for layer in pre_trained_model.layers:
    layer.trainable = True
    
last_layer = pre_trained_model.get_layer("activation_74")
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 16, 16, 256)


In [6]:
x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(6, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = 'Adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

#model.summary()

In [7]:
history = model.fit(train_generator,
                    epochs=100,
                    validation_data = validation_generator)

Epoch 1/100
439/439 [==============================] - 150s 341ms/step - loss: 1.0433 - accuracy: 0.6231 - val_loss: 0.4784 - val_accuracy: 0.8463
Epoch 2/100
439/439 [==============================] - 129s 293ms/step - loss: 0.5511 - accuracy: 0.8138 - val_loss: 0.4483 - val_accuracy: 0.8390
Epoch 3/100
439/439 [==============================] - 130s 296ms/step - loss: 0.4648 - accuracy: 0.8449 - val_loss: 0.4499 - val_accuracy: 0.8533
Epoch 4/100
439/439 [==============================] - 129s 294ms/step - loss: 0.4231 - accuracy: 0.8594 - val_loss: 0.3786 - val_accuracy: 0.8690
Epoch 5/100
439/439 [==============================] - 127s 290ms/step - loss: 0.4078 - accuracy: 0.8648 - val_loss: 0.3508 - val_accuracy: 0.8650
Epoch 6/100
439/439 [==============================] - 128s 292ms/step - loss: 0.3833 - accuracy: 0.8690 - val_loss: 0.4092 - val_accuracy: 0.8730
Epoch 7/100
439/439 [==============================] - 129s 295ms/step - loss: 0.3745 - accuracy: 0.8748 - val_loss: 0